# Setup

Importing file

In [35]:
from google.colab import files


text_file = files.upload()

Saving Test_text_file.txt to Test_text_file.txt


1. Computing an estimate of the
probabilities of the different English characters (symbols) in this text
file

In [36]:
# saving the text in the file in a string
text = text_file["Test_text_file.txt"]

text = str(text)
text = text[2:len(text)-1]

# create dictionary with character and their coressponding probability
char_probability = {}
for i in range(97, 123):
    char_probability[chr(i)] = 0
char_probability['('] = 0
char_probability[')'] = 0
char_probability['.'] = 0
char_probability[','] = 0
char_probability['/'] = 0
char_probability['-'] = 0
char_probability[' '] = 0

for c in text:
    char_probability[c] += 1
for x in char_probability:
    char_probability[x] = char_probability[x]/len(text)
print(char_probability)

{'a': 0.07193732193732194, 'b': 0.012108262108262107, 'c': 0.026353276353276354, 'd': 0.045584045584045586, 'e': 0.10398860398860399, 'f': 0.013532763532763533, 'g': 0.014245014245014245, 'h': 0.02207977207977208, 'i': 0.06054131054131054, 'j': 0.0007122507122507123, 'k': 0.0007122507122507123, 'l': 0.04202279202279202, 'm': 0.021367521367521368, 'n': 0.0641025641025641, 'o': 0.06267806267806268, 'p': 0.037037037037037035, 'q': 0.0007122507122507123, 'r': 0.05982905982905983, 's': 0.05626780626780627, 't': 0.07193732193732194, 'u': 0.018518518518518517, 'v': 0.009259259259259259, 'w': 0.007834757834757835, 'x': 0.004985754985754986, 'y': 0.00641025641025641, 'z': 0.002136752136752137, '(': 0.002136752136752137, ')': 0.002136752136752137, '.': 0.004273504273504274, ',': 0.007834757834757835, '/': 0.0007122507122507123, '-': 0.011396011396011397, ' ': 0.1346153846153846}


2. Calculating the entropy

In [37]:
import math
entropy = 0
for c in char_probability:
    entropy += -char_probability[c] * math.log2(char_probability[c])

entropy

4.25701056473807

3. Calculating the number of bits/symbol in a fixed length
code and its effeciency

In [38]:
bits_fixed_length =  math.ceil(math.log2(len(char_probability)))
effeciency_fixed_length = entropy/bits_fixed_length
print("fixed length code\t "+ str(bits_fixed_length) + " bits/symbol\n", "effeciency\t\t", effeciency_fixed_length*100, " %")

fixed length code	 6 bits/symbol
 effeciency		 70.95017607896783  %


# Huffman

4. Encoder

In [39]:
# creating class for tree node
class Node:
    def __init__(self, char=None, probability=0):
        self.char = char
        self.probability = probability
        self.left = None
        self.right = None

# initializing nodes
tree = []
for x in char_probability:
    tree.append(Node(x, char_probability[x]))

# creating tree
while(len(tree) > 1):
    tree.sort(key = lambda p: -p.probability)  # sort probabilities descendingly
    left = tree.pop()
    right = tree.pop()
    node = Node('*',left.probability + right.probability)
    node.right = right
    node.left = left
    tree.append(node)



# creating code
char_code = {}
for x in char_probability:
    char_code[x] = ""

def code(node, node_code):
    if node:
        if node.char != '*':
            char_code[node.char] = node_code
        code(node.left, node_code + "0")
        code(node.right, node_code  + "1")

code(tree[0],'')

def encode_huffman(text):
    encoded_text = ""
    for s in text:
        encoded_text += char_code[s]

    return encoded_text

huffman_encoded_text = encode_huffman(text)


5. Decoder

In [40]:
# Decode the Huffman encoded text directly
def decode_huffman(encoded_text, root):
    decoded_text = ""
    current_node = root

    for bit in encoded_text:
        if bit == '0':
            current_node = current_node.left
        elif bit == '1':
            current_node = current_node.right

        # If a leaf node is reached, append the character and reset to root
        if current_node.char != '*':  # Leaf node
            decoded_text += current_node.char
            current_node = root  # Reset to root for the next character

    return decoded_text

# Decode the encoded text
decoded_text = decode_huffman(huffman_encoded_text, tree[0])

# Print the decoded text
print("Decoded Text:")
print(decoded_text)



Decoded Text:
in this paper, a novel decorrelation-based concurrent digital predistortion (dpd) solution is proposed for dual-band transmitters (tx) employing a single wideband power amplifier (pa), and utilizing just a single feedback receiver path. the proposed decorrelation-based parameter learning solution is both flexible and simple, and operates in a closed-loop manner, opposed to the widely applied indirect learning architecture. the proposed decorrelation-based learning and dpd processing can also be effectively applied to more ordinary single-band transmissions, as well as generalized to more than two transmit bands. through a comprehensive analysis covering both the dpd parameter learning and the main path processing, it is shown that the complexity of the proposed concurrent dpd is substantially lower compared with the other state-of-the-art concurrent dpd methods. extensive set of quantitative simulation and rf measurement results are also presented, using a base-station pa

In [41]:
with open("decode_file.txt", "w") as file:
    file.write(decoded_text)

comparing decoded stream with the original stream

In [42]:

def CompareTexts(text1, text2):
  if len(text1) != len(text2):
    return False
  for i in range(0, len(text1)):
    if text1[i] != text2[i]:
      print(i)
      return False
  return True

print("Texts are the same") if CompareTexts(text, decoded_text) else print("Texts are not the same")

Texts are the same


6. Calculating the efficiency of the Huffman code

In [43]:
def CalculateEfficiencyHuffman():
  average_code_length = 0
  for code in char_code:
    average_code_length += len(char_code[code]) * char_probability[code]
  return entropy / average_code_length

print("Huffman code efficiency: ", CalculateEfficiencyHuffman())

Huffman code efficiency:  0.995476820934752


# Shannon-Fano

7. Encoder

In [44]:
from pprint import pprint
def ShannonFanoEncoder(symbols):
  codes = {}
  assignCodes(symbols, "", codes)
  return codes

def assignCodes(symbols, prefix, codes):
    if len(symbols) == 1:
        codes[symbols[0][0]] = prefix
        return

    splitPoint = findSplitPoint(symbols)
    firstGroup = symbols[:splitPoint + 1]
    secondGroup = symbols[splitPoint + 1:]

    assignCodes(firstGroup, prefix + "0", codes)
    assignCodes(secondGroup, prefix + "1", codes)


def findSplitPoint(symbols):
    totalFrequency = 0
    for i in range(0, len(symbols)):
        totalFrequency += symbols[i][1]

    cumulativeFrequency = 0
    half_total = totalFrequency / 2
    for i in range(0, len(symbols)):
        cumulativeFrequency += symbols[i][1]
        if cumulativeFrequency >= half_total:
          return i if (abs(cumulativeFrequency - half_total) < abs(cumulativeFrequency - (half_total + symbols[i][1]))) else i - 1


    return len(symbols) - 1

def EncodeText(text, codes):
  encoded_text = ""
  for char in text:
    encoded_text += codes[char]
  return encoded_text


char_probability1 = {
    'a': 0.3,
    'b': 0.25,
    'c': 0.2,
    'd': 0.15,
    'e': 0.1
}
text1 = "abcdabcdabdd"

sorted_char_probability = sorted(char_probability.items(), key=lambda x: x[1],  reverse=True)
codes = ShannonFanoEncoder(sorted_char_probability)
encoded_text = EncodeText(text, codes)

Decoder

In [45]:
def ShannonFanoDecoder(encoded_text, codes):
    decoded_text = ""
    current_code = ""
    for char in encoded_text:
        current_code += char
        for code in codes:
            if codes[code] == current_code:
                decoded_text += code
                current_code = ""
                break
    return decoded_text


decoded_text = ShannonFanoDecoder(encoded_text, codes)


comparing decoded stream with the original stream

In [46]:
def CompareTexts(text1, text2):
  if len(text1) != len(text2):
    return False
  for i in range(0, len(text1)):
    if text1[i] != text2[i]:
      print(i)
      return False
  return True

print("Texts are the same") if CompareTexts(text, decoded_text) else print("Texts are not the same")

Texts are the same


Calculating the efficiency of the Shannon-Fano code

In [47]:
def CalculateEfficiencyShannon():
  average_code_length = 0
  for code in codes:
    average_code_length += len(codes[code]) * char_probability[code]
  return entropy / average_code_length

print("Shannon-Fano code efficiency: ", CalculateEfficiencyShannon())

Shannon-Fano code efficiency:  0.9948140534108271


**8-Efficiency Comparison: Shannon-Fanovs. Huffman Codes**

In [48]:
# Comparison Function using pre-computed variables
def compare_coding_methods(fixed_avg_length, huffman_avg_length, shannon_fano_avg_length,
                           fixed_efficiency, huffman_efficiency, shannon_fano_efficiency):
    # Display Results
    print(f"{'Method':<15} {'Average Length':<20} {'Efficiency (%)':<20}")
    print(f"{'Fixed-Length':<15} {fixed_avg_length:<20.4f} {fixed_efficiency * 100:<20.2f}")
    print(f"{'Huffman':<15} {huffman_avg_length:<20.4f} {huffman_efficiency * 100:<20.2f}")
    print(f"{'Shannon-Fano':<15} {shannon_fano_avg_length:<20.4f} {shannon_fano_efficiency * 100:<20.2f}")

# Call the function with the pre-computed values
compare_coding_methods(bits_fixed_length, math.ceil(CalculateEfficiencyHuffman()*entropy), math.ceil(CalculateEfficiencyShannon()*entropy),
                       effeciency_fixed_length, CalculateEfficiencyHuffman(), CalculateEfficiencyShannon())


Method          Average Length       Efficiency (%)      
Fixed-Length    6.0000               70.95               
Huffman         5.0000               99.55               
Shannon-Fano    5.0000               99.48               
